In [1]:
# Set logging
import logging

logging.basicConfig(level=logging.DEBUG)

# Set the GPU_ENABLED Flag to True
import os
os.environ['GPU_ENABLED'] = 'True'
print(os.environ['GPU_ENABLED'])  # Should print 'True'

from story_wrapper.models.story import Story
from story_wrapper.data_loaders.gutenberg import Gutenberg

g_corpus = Gutenberg()
book = g_corpus.get_book_object(13034)

story = Story(book.paragraphs)

True


INFO:root:Indexing Gutenberg corpus.
INFO:root:Reading metadata from /home/ben/Projects/story-wrapper/src/story_wrapper/data_loaders/indexes/md.pickle.gz
INFO:root:Indexing complete.
INFO:root:Number of books in Gutenberg corpus: 69478
 at line 8197 line: End of Project Gutenberg's Mary Minds Her Business, by George Weston
DEBUG:root:TOC locations to be deleted: []
INFO:root:Heading locations: [292, 408, 512, 625, 942, 1108, 1255, 1343, 1434, 1643, 1896, 2151, 2367, 2539, 2676, 2937, 3082, 3212, 3416, 3516, 3717, 4010, 4140, 4388, 4616, 5364, 5534, 6361, 6419, 6807, 6973, 7201, 7349, 7670, 7815, 7980, 8086, 8197]
INFO:root:Headings: ['CHAPTER I\r', 'CHAPTER II\r', 'CHAPTER III\r', 'CHAPTER IV\r', 'CHAPTER V\r', 'CHAPTER VI\r', 'CHAPTER VII\r', 'CHAPTER VIII\r', 'CHAPTER IX\r', 'CHAPTER X\r', 'CHAPTER XI\r', 'CHAPTER XII\r', 'CHAPTER XIII\r', 'CHAPTER XIV\r', 'CHAPTER XV\r', 'CHAPTER XVI\r', 'CHAPTER XVII\r', 'CHAPTER XVIII\r', 'CHAPTER XIX\r', 'CHAPTER XX\r', 'CHAPTER XXI\r', 'CHAPTER 

In [2]:
story.potential_characters

{}

In [3]:
print(story.names_to_process)

['patty', 'cordelia', 'josiah', 'patricia', 'spencers', 'spencer', 'spicers', 'abner spicer', 'ezra babcock', 'josiah spencer', 'pearsons', 'martha berger', 'martha', 'abner spencer', 'sarah', 'barbara fritchie', 'patrick henry', 'basil', 'angel of death', 'angel of life', 'mary', 'dauphin of', 'maynard', "t'delia", 'eve', 'woodward', 'stanley woodward', 'cousin stanley', 'stanley', 'burdon', 'chip', 'archey forbes', 'archey', 'spicer', 'helen', 'bob mcallister', 'robert', 'mcallister', "ma'm", 'trus', 'mon dieu', 'cutler', 'parsons', 'st. cecilia', 'becky', 'vera', 'marx', 'wally', 'james', 'brad', 'wally cabot', 'paul', 'walter cabot', 'nita', 'juanita', 'garde', 'will thompson', 'punch', 'washington', 'fulton', 'morse', 'columbus', 'george washington', 'ridge', 'shakespeare', 'mary spencer', 'm. spencer', 'matthew', 'mark', 'michael', 'malachi', 'uncle stanley', 'destiny', 'burdon woodward', 'raffles', 'steerforth', 'david copperfield', 'morgan', 'kidd', 'black jack', 'dora', "d'art

In [4]:
story.process_characters()

{8727801707848: <Character - ID: 8727801707848; name: 'abner spicer'; no. of occurrences: 3; first name: 'abner'; surname: 'spicer'; middle names: []; aliases: []>,
 8727613624579: <Character - ID: 8727613624579; name: 'ezra babcock'; no. of occurrences: 2; first name: 'ezra'; surname: 'babcock'; middle names: []; aliases: []>,
 8727802834845: <Character - ID: 8727802834845; name: 'josiah spencer'; no. of occurrences: 67; first name: 'josiah'; surname: 'spencer'; middle names: []; aliases: []>,
 8727802834545: <Character - ID: 8727802834545; name: 'martha berger'; no. of occurrences: 21; first name: 'martha'; surname: 'berger'; middle names: []; aliases: []>,
 8727802834557: <Character - ID: 8727802834557; name: 'abner spencer'; no. of occurrences: 1; first name: 'abner'; surname: 'spencer'; middle names: []; aliases: []>,
 8727802834677: <Character - ID: 8727802834677; name: 'barbara fritchie'; no. of occurrences: 1; first name: 'barbara'; surname: 'fritchie'; middle names: []; aliase

In [8]:
print(sorted(story.names_to_process))

['basil', 'batuchka', 'becky', 'bill', 'bols', 'brad', 'buckingham', 'cabot', 'chip', 'columbus', 'coppelia', 'cordelia', 'cutler', "d'artagnan", 'delia', 'delibes', 'destiny', 'dora', 'doris', 'edsol', 'eve', 'fanny', 'fie', 'forbes', 'fulton', 'garde', 'hagar', 'haskins', 'hutch', 'hutchins', 'ishmaels', 'james', 'jason', 'jimmy', 'jingo', 'joe', 'joseph', 'josiahs', 'jove', 'juanita', 'judas', 'kidd', 'lorelei', 'mac', 'malachi', 'mark', 'mars', 'marsh', 'marx', 'mary', 'matthew', 'micawber', 'michael', 'molls', 'morgan', 'morse', 'moses', 'ned', 'nina', 'nita', 'parsons', 'patricia', 'patty', 'pearsons', 'punch', 'raffles', 'repetti', 'ridge', 'robert', 'rosa', 'sarah', 'shakespeare', 'spencer', 'spencers', 'spicers', 'steerforth', 'strauss', "t'delia", 'trus', 'vera', 'wally', 'woodward', 'worth']


* If a single name does not match any of the existing first names as a firstname or nickname we can add as a character without a surname.
    * But how do we know first names from surnames?
    
    
* We can match on initial:
    * If there is a surname match within character combinations and the first name has an initial and the initial matches the first letter of the first name.

In [21]:
from collections import defaultdict
from story_wrapper.models.story import nn

characters = story.potential_characters
for name in story.names_to_process:
    character_match = defaultdict(list)
    for char in characters.values():
        existing_names = {char.surname, char.firstname} | {alias for alias in char.aliases}
        # Add also potential nicknames
        existing_names.update(nn.nicknames_of(char.firstname))
        # Also check for plurals - TODO make as function with similar from merge_by_plural
        # Check that existing surname is not a plural then add the plural form
        if not char.surname.endswith("s"):
            if not char.occurrences[0].span[-1].tag_ == "NNPS":
                # Add with an s to compare
                existing_names.add(char.surname + "s")
        print(f"Comparing: '{name}' to {existing_names}")
        if name in existing_names:
            character_match[name].append(char.id)
    print(character_match[name])
    if len(character_match[name]) == 0:
        print(f"{name} has no matches with existing potential characters")
    else:
        print(f"{name} has matches: {[characters[c] for c in character_match[name]]}")

Comparing: 'patty' to {'spicers', 'spicer', 'abner', 'ab'}
Comparing: 'patty' to {'ezra', 'babcocks', 'ez', 'babcock'}
Comparing: 'patty' to {'josiah', 'spencers', 'spencer', 'jos'}
Comparing: 'patty' to {'mat', 'berger', 'martha', 'marty', 'patty', 'patsy', 'mattie', 'bergers'}
Comparing: 'patty' to {'abner', 'spencers', 'spencer', 'ab'}
Comparing: 'patty' to {'barbara', 'bab', 'babs', 'bobbie', 'fritchies', 'barby', 'fritchie'}
Comparing: 'patty' to {'patrick', 'henry', 'patsy', 'peter', 'pate', 'pat', 'henrys', 'paddy'}
Comparing: 'patty' to {'death', 'angel', 'deaths'}
Comparing: 'patty' to {'angel', 'lifes', 'life'}
Comparing: 'patty' to {'of', 'dauphin', 'ofs'}
Comparing: 'patty' to {'stanley', 'woodward', 'woodwards'}
Comparing: 'patty' to {'stanley', 'cousin', 'stanleys'}
Comparing: 'patty' to {'forbes', 'archey'}
Comparing: 'patty' to {'robert', 'bob', 'mcallisters', 'mcallister', 'rob'}
Comparing: 'patty' to {'dieu', 'mon', 'dieus'}
Comparing: 'patty' to {'st.', 'cecilias', '

In [6]:
story.count_characters()

Counter({'Mary': 582,
         'Helen': 178,
         'Wally': 112,
         'Cordelia': 73,
         'Archey': 69,
         'Burdon': 65,
         'Patty': 63,
         'Josiah': 60,
         'Stanley': 59,
         'Paul': 52,
         'Spencer': 40,
         'Cutler': 27,
         'Martha': 20,
         'Hutchins': 15,
         'Burdon Woodward': 14,
         'Joe': 14,
         'Woodward': 12,
         'Maynard': 11,
         'Parsons': 10,
         'Wally Cabot': 10,
         'Mac': 10,
         'Spencers': 9,
         'Archey Forbes': 9,
         'Ridge': 7,
         'Kelly': 7,
         'Rosa': 6,
         'Strauss': 6,
         'Josiah Spencer': 4,
         'Stanley Woodward': 4,
         'Sam Reisinger': 4,
         'Marsh': 4,
         'Vera': 3,
         'Washington': 3,
         'MacPherson': 3,
         "Ma'm Maynard's": 3,
         'Josiah Spencers': 3,
         'Haskins': 3,
         'Repetti': 3,
         'Forbes': 3,
         'Patricia': 2,
         'Pearsons': 2,
    

In [7]:
for char in story.potential_characters.values():
    print(char)
    for o in char.occurrences:
        print(o)
        print(story.text[o.location_number])
        print("---")
    print("\n******\n")

<Character - ID: 8727801707848; name: 'abner spicer'; no. of occurrences: 3; first name: 'abner'; surname: 'spicer'; middle names: []; aliases: []>
<Occurrence - text index: 14; span: Abner Spicer>
"I wonder if Josiah--is too old--to marry again! Of course," she hurriedly added, "he is fifty-two--but it seems to me that one of the Spicers--I think it was Captain Abner Spicer--had children until he was sixty--although by a younger wife, of course."
---
<Occurrence - text index: 195; span: Spicer>
In looking up the genealogy of the Spicer family, Miss Patty discovered that a distant relative in Charleston had just died, leaving a daughter behind him--an orphan--who was a year older than Mary. Correspondence finally led Miss Patty to make the journey, and when she returned she brought with her a dark-eyed girl who might have been the very spirit of youthful romance.
---
<Occurrence - text index: 310; span: Spicer>
To college! A Spencer girl--or a Spicer--going to college! Miss Cordelia ga

In [10]:
# Sort the dictionary by the length of each Character object's "occurrences" field
sorted_characters = sorted(story.potential_characters.values(), key=lambda x: len(x.occurrences), reverse=True)

In [11]:
sorted_characters

[<Character - ID: 8794575830007; name: 'helen cabot'; no. of occurrences: 179; first name: 'helen'; surname: 'cabot'; middle names: []; aliases: []>,
 <Character - ID: 8794575829959; name: 'burdon woodward'; no. of occurrences: 81; first name: 'burdon'; surname: 'woodward'; middle names: []; aliases: []>,
 <Character - ID: 8794575829932; name: 'archey forbes'; no. of occurrences: 78; first name: 'archey'; surname: 'forbes'; middle names: []; aliases: []>,
 <Character - ID: 8795007771129; name: 'josiah spencer'; no. of occurrences: 67; first name: 'josiah'; surname: 'spencer'; middle names: []; aliases: []>,
 <Character - ID: 8794575829899; name: 'stanley woodward'; no. of occurrences: 63; first name: 'stanley'; surname: 'woodward'; middle names: []; aliases: []>,
 <Character - ID: 8794575830013; name: 'paul spencer'; no. of occurrences: 54; first name: 'paul'; surname: 'spencer'; middle names: []; aliases: []>,
 <Character - ID: 8794575829920; name: 'martha berger'; no. of occurrences: